In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from pathlib import Path

DATA_DIR = Path("cv_tracking_data")
DATA_DIR.mkdir(exist_ok=True)

!wget -q -O {DATA_DIR}/vtest.avi https://raw.githubusercontent.com/opencv/opencv/master/samples/data/vtest.avi


cap = cv2.VideoCapture(str(DATA_DIR/'vtest.avi'))
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

p0 = cv2.goodFeaturesToTrack(old_gray, maxCorners=100, qualityLevel=0.3, minDistance=7)

lk_params = dict(winSize=(15,15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT, 10, 0.03))

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret or frame_count>60:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    if p1 is None:
        break
    good_new = p1[st==1]
    good_old = p0[st==1]


    for (new, old) in zip(good_new, good_old):
        a, b = new.ravel()
        c, d = old.ravel()
        cv2.line(frame, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        cv2.circle(frame, (int(a), int(b)), 3, (0, 0, 255), -1)

    cv2_imshow(frame)

    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
    frame_count += 1

cap.release()
cv2.destroyAllWindows()
print("Optical flow demo finished")